## import and load

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import notebook
import datetime

import xgboost as xgb
from sklearn.model_selection import train_test_split
import lightgbm

#한글 깨짐방지
plt.rc('font',family = 'Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False

C:\Users\User\Anaconda3\lib\site-packages\distributed\config.py:63: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config.update(yaml.load(text) or {})


In [2]:
path = './data/'
train = pd.read_csv(path + 'train.csv')
test = pd.read_csv(path + 'test.csv')
submission  = pd.read_csv(path + 'submission.csv')

## data setting

In [3]:
#time, installment, days_of_week, card_id, holyday는 필요없다 생각하여 글쓴이는 제거
train = train.drop(columns=['time', 'installments', 'days_of_week', 'card_id', 'holyday'])
test = test.drop(columns=['time', 'installments', 'days_of_week', 'card_id', 'holyday'])

In [4]:
# 'date','store_id'로 결합
train = train.groupby(['date','store_id']).agg({'amount':'sum'}).reset_index()
test = test.groupby(['date','store_id']).agg({'amount':'sum'}).reset_index()

In [5]:
#이러한 형태로 변환됨
train.head(1)

,date,store_id,amount
0,2016-08-01,2,2372


In [6]:
#date전환
train['date'] = pd.to_datetime(train['date'],infer_datetime_format= True)
test['date'] = pd.to_datetime(test['date'],infer_datetime_format= True)

In [7]:
# 나중을 위한 복제
train['temp_date'] = train['date']
test['temp_date'] = test['date']

In [8]:
# 날짜를 인덱스로 설정
train.set_index("date",inplace=True)
test.set_index("date",inplace=True)

### 일부제거

#### train
1. 거래기록이 160일 이하인 경우 삭제 -> 대회 목적이 100일 이후를 예측하는 것이므로 60일은 train(X), 100일은 predict(y)에 사용
    * 160일이 가장 이상적이라고 고려함
2. 2018-07-31 주변 5일 내 거래가 없으면 폐업으로 추정, 2018-07-31은 꼭 있어야하는 데이터

In [9]:
counter = 0
limit = 160

print("Before removing stores (due to limit): ", train.shape)

#160 이상이면 카운트를 올리고, 아니라면 제외함
for store_id in notebook.tqdm(range(train['store_id'].max()+1)):
    if train.loc[train['store_id'] == store_id, 'store_id' ].count() >= limit:
        counter += 1
    else :
        train = train[train.store_id !=store_id]

print(f"Total # of stores that exceeds {limit} is {counter}")
print("After removing stores (due to limit): ", train.shape)

Before removing stores (due to limit):  (480160, 3)



Total # of stores that exceeds 160 is 997
After removing stores (due to limit):  (432671, 3)


In [10]:
#  2018-07-31 주변 5일 내 거래가 없으면 폐업으로 추정, 2018-07-31은 꼭 있어야하는 데이터
def keep_alive_store(df):
    store_id_list = df.store_id.unique()
    yes, no = 0, 0
    
    t2 = datetime.datetime.strptime('2018-07-31', '%Y-%m-%d')
    
    for store_id in store_id_list:
        if df.loc[df.store_id == store_id,'temp_date'].iloc[-1] == t2:
            yes += 1
        else:
            t1 = df.loc[df.store_id == store_id,'temp_date'].iloc[-1]
            difference = t2 - t1
            if difference.days <=5:
                yes += 1
            else :
                no += 1
                df = df[df.store_id != store_id]
    print(f"# of train store open/out of business: {yes}, {no}")
    return df

In [11]:
train = keep_alive_store(train)

# of train store open/out of business: 906, 91


#### test
1. 2018-03-31 주변 7일 거래기록이 없으면 폐업으로 가정 
  - 197번 매장을 폐업으로 추정

In [12]:
store_id_list = test.store_id.unique()
yes, no = 0, 0
closed_test_store = []
t2 = datetime.datetime.strptime('2018-03-01',"%Y-%m-%d")

for store_id in store_id_list:    
    t1 = test[test.store_id == store_id]['temp_date'].iloc[-1]
    differ = t2 - t1
    
    if differ.days <= 7 :
        yes += 1 
    else :
        no += 1
        print(test[test.store_id == store_id].iloc[-1]['temp_date'])
        closed_test_store.append(store_id)
print(f"# of test store open/out of business: {yes}, {no}")

2018-01-17 00:00:00
# of test store open/out of business: 199, 1


### reform

In [14]:
def reform_data(df, isTrain):
    store_id_list = df.store_id.unique()
    x_array = []
    for store_id in store_id_list:
        store = df[df.store_id == store_id]
        store = store.asfreq('D',fill_value = 0) #일자별로 열추가 
        store['temp_date'] = store.index
        store['store_id'] = store_id
        
        # 7일, 15일, 30일간의 이동평균 계산 
        store['MA7'] = store['amount'].rolling('7D').mean()
        store['MA15'] = store['amount'].rolling('15D').mean()
        store['MA30'] = store['amount'].rolling('30D').mean()
        
        # train set인 경우 마지막 100일은 y값으로 저장되어야함
        # store_x는 100일전까지 기간 데이터 
        if isTrain:
            store_y = store.last("100D")
            y = store_y.amount.sum()
            store_x = store[store.temp_date < store_y.iloc[0].temp_date]
        #test는 전체
        else : 
            y = 0
            store_x = store[:]
            
        new_data = []
        new_data.append(store_id)
        new_data.append(y)
        
        new_data.append(store_x.amount.mean())
        new_data.append(store_x.amount.median())
        
        new_data.append(store_x.last('7D').amount.mean())
        new_data.append(store_x.last('15D').amount.mean())
        new_data.append(store_x.last('30D').amount.mean())
        
        new_data.append(store_x.last('7D').amount.median())
        new_data.append(store_x.last('15D').amount.median())
        new_data.append(store_x.last('30D').amount.median())
        
        new_data.append(store_x.last('7D').amount.sum())
        new_data.append(store_x.last('15D').amount.sum())
        new_data.append(store_x.last('30D').amount.sum())
        
        new_data.append(store_x.last('7D').MA7.mean())
        new_data.append(store_x.last('15D').MA7.mean())
        new_data.append(store_x.last('30D').MA7.mean())
        
        new_data.append(store_x.last('7D').MA15.mean())
        new_data.append(store_x.last('15D').MA15.mean())
        new_data.append(store_x.last('30D').MA15.mean())
        
        new_data.append(store_x.last('7D').MA30.mean())
        new_data.append(store_x.last('15D').MA30.mean())
        new_data.append(store_x.last('30D').MA30.mean())
        
        x_array.append(new_data)
    
    return x_array

In [15]:
reformed_train = reform_data(train, True) # train data with new values
reformed_test = reform_data(test, False) # test data with new values

In [16]:
#컬럼이름 부여
train_fin = pd.DataFrame(reformed_train, columns=['store_id', 'y', 'mean', 'median', '7mean', '15mean', '30mean', 
                                         '7median', '15median', '30median',  '7sum', '15sum', '30sum', 
                                         '7ma7mean', '15ma7mean', '30ma7mean',  '7ma15mean', '15ma15mean',
                                         '30ma15mean',  '7ma30mean', '15ma30mean', '30ma30mean'])
test_fin = pd.DataFrame(reformed_test, columns=['store_id', 'y', 'mean', 'median', '7mean', '15mean', '30mean', 
                                         '7median', '15median', '30median',  '7sum', '15sum', '30sum', 
                                         '7ma7mean', '15ma7mean', '30ma7mean',  '7ma15mean', '15ma15mean',
                                         '30ma15mean',  '7ma30mean', '15ma30mean', '30ma30mean'])

In [17]:
train_fin.shape, test_fin.shape

((906, 22), (200, 22))

In [19]:
train_fin

,store_id,y,mean,median,7mean,15mean,30mean,7median,15median,30median,...,30sum,7ma7mean,15ma7mean,30ma7mean,7ma15mean,15ma15mean,30ma15mean,7ma30mean,15ma30mean,30ma30mean
0,2,186853,2266.995238,2267.0,2207.714286,2209.466667,2272.233333,2174.0,2174.0,2293.0,...,68167,2201.020408,2203.819048,2297.928571,2203.819048,2233.044444,2319.146667,2297.928571,2319.146667,2344.041111
1,6,219761,1857.495238,1080.0,2076.428571,2359.133333,1970.233333,1670.0,1670.0,1622.5,...,59107,2703.020408,2448.704762,2690.871429,2448.704762,2061.066667,2833.988889,2690.871429,2833.988889,3410.305556
2,8,79695,691.809524,500.0,623.571429,704.333333,650.666667,260.0,450.0,475.0,...,19520,570.102041,659.047619,591.166667,659.047619,635.622222,629.444444,591.166667,629.444444,664.450000
3,12,147744,1224.731746,898.5,1995.000000,1524.333333,1118.200000,2145.0,1155.0,990.0,...,33546,1772.346939,1273.571429,971.723810,1273.571429,990.880000,860.555556,971.723810,860.555556,824.680000
4,14,17253,309.682035,150.0,267.142857,218.733333,340.766667,195.0,187.0,153.5,...,10223,252.836735,471.485714,341.904762,471.485714,465.400000,346.037778,341.904762,346.037778,305.583333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901,242,55006,512.181818,395.0,758.714286,580.733333,608.233333,530.0,505.0,454.5,...,18247,421.469388,537.009524,530.876190,537.009524,579.191111,554.102222,530.876190,554.102222,544.604444
902,120,176547,1418.546667,1238.0,1524.000000,1503.066667,1360.633333,1586.0,1311.0,1109.5,...,40819,1416.367347,1414.380952,1278.057143,1414.380952,1344.075556,1251.404444,1278.057143,1251.404444,1253.010000
903,1782,101682,1727.285714,1625.0,1644.285714,1474.000000,1714.333333,1680.0,1505.0,1595.0,...,51430,1531.428571,1402.190476,1753.809524,1402.190476,1482.822222,1837.511111,1753.809524,1837.511111,1986.222222
904,904,62155,332.205882,287.5,577.857143,506.000000,470.333333,560.0,510.0,425.0,...,14110,509.183673,477.523810,425.833333,477.523810,454.933333,407.855556,425.833333,407.855556,361.844444


### modeling
* xgboost를 활용, test분할은 0.1만

In [18]:
col = [i for i in train_fin.columns if i not in ['store_id','y']]
y = train_fin['y'].values

In [83]:
train_X, test_X, train_y, test_y = train_test_split(train_fin[col],y, test_size=0.1, random_state=2018)

#### lgbm

In [105]:
lgbm = lightgbm.LGBMRegressor(learning_rate=0.1, n_estimators=30000)

lgbm.fit(train_X,train_y,eval_set=[(train_X,train_y),(test_X,test_y)], eval_names=['train','test'],eval_metric='rmse',verbose=100, early_stopping_rounds= 500)

In [110]:
y_pred = lgbm.predict(test_X)

In [111]:
from sklearn.metrics import mean_squared_error

In [116]:
np.sqrt(mean_squared_error(test_y, y_pred))

30120.484413714657

In [122]:
lgbm = lightgbm.LGBMRegressor(learning_rate=0.1, n_estimators=1000)

In [123]:
lgbm.fit(train_fin,y,eval_metric='rmse',verbose=100, eval_set=[(train_fin,y)],early_stopping_rounds=300)

Training until validation scores don't improve for 300 rounds
[100]	training's rmse: 63549.2	training's l2: 4.0385e+09
[200]	training's rmse: 53519.5	training's l2: 2.86433e+09
[300]	training's rmse: 45606.6	training's l2: 2.07996e+09
[400]	training's rmse: 37710.8	training's l2: 1.4221e+09
[500]	training's rmse: 31846.2	training's l2: 1.01418e+09
[600]	training's rmse: 27384.1	training's l2: 7.49888e+08
[700]	training's rmse: 23650.5	training's l2: 5.59346e+08
[800]	training's rmse: 20674.1	training's l2: 4.27418e+08
[900]	training's rmse: 18293.3	training's l2: 3.34646e+08
[1000]	training's rmse: 16146	training's l2: 2.60692e+08
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 16146	training's l2: 2.60692e+08


LGBMRegressor(n_estimators=1000)

In [124]:
y_pred = lgbm.predict(test_fin)

#### XGB

In [128]:
def XGB_regressor(train_X, train_y, test_X, test_y, metric, feature_name = None, seed_val = 2018, num_rounds = 3000):
    params = {'objective' : 'reg:squarederror',
              'learning_rate' : 0.05,
              'max_depth' : 10,
              'eval_metric': metric,#??? rmse가 아니고??
              'min_child_weight': 1,
              'subsample': 0.7,
              'colsample_bytree' : 0.7,
              'seed': seed_val}
    plst = list(params.items())
    
    xgtrain = xgb.DMatrix(train_X, label = train_y)
    
    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label = test_y)
        watchlist = [ (xgtrain, 'train'), (xgtest, 'test')]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds= 300,verbose_eval= 100)
    else :
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)
        
    return model

In [129]:
model = XGB_regressor(train_X, train_y, test_X, test_y,'mae')

[0]	train-mae:70772.46875	test-mae:70710.32031
Multiple eval metrics have been passed: 'test-mae' will be used for early stopping.

Will train until test-mae hasn't improved in 300 rounds.
[100]	train-mae:3777.10156	test-mae:13821.57812
[200]	train-mae:1662.26575	test-mae:13788.00781
[300]	train-mae:732.37915	test-mae:13782.84082
[400]	train-mae:325.06516	test-mae:13810.90527
Stopping. Best iteration:
[155]	train-mae:2382.86499	test-mae:13696.49609



In [86]:
model = XGB_regressor(train_X, train_y, test_X, test_y, 'rmse')

[0]	train-rmse:142892.68750	test-rmse:116447.28125
Multiple eval metrics have been passed: 'test-rmse' will be used for early stopping.

Will train until test-rmse hasn't improved in 300 rounds.
[100]	train-rmse:12702.62305	test-rmse:30103.52344
[200]	train-rmse:2737.90820	test-rmse:29411.41211
[300]	train-rmse:989.69238	test-rmse:29387.22852
[400]	train-rmse:435.31125	test-rmse:29396.12109
[500]	train-rmse:190.40916	test-rmse:29408.71875
Stopping. Best iteration:
[215]	train-rmse:2302.21118	test-rmse:29350.26953



In [131]:
##??? sorting을 이제한다고???
test_fin = test_fin.sort_values(by='store_id')

In [143]:
#몇가지 조정을 더 거친후 제출할 답변임
y_pred = model.predict(xgb.DMatrix(test_fin[col]), ntree_limit = model.best_ntree_limit)

In [88]:
# 이상해서 한번 제출해봄 -> 오버피팅 과도하게되고 결과 안 좋음

score = 29350.26953
submission['total_sales'] = y_pred

today = datetime.datetime.today().date()
os.makedirs('./submission/',exist_ok=True)
submission.to_csv(f'./submission/submission_{today}_xgb_{score}_rmse.csv',index=False)

### 추가조정
* 몇몇 상점은 2~3월에 폐업했을 것임

In [156]:
store_id_list = test.store_id.unique()
store_id_list.sort()

feb_march = []
for store_id in store_id_list:
    mini = test[test.store_id == store_id]
    march = mini[mini.temp_date >= datetime.datetime(2018,3,1)]['temp_date'].count()
    feb = mini[mini.temp_date >= datetime.datetime(2018,2,1)]['temp_date'].count() - march 
    feb_march.append((march+feb)/2/31) 

In [157]:
#2-3월 확인 전에 이전에 발견한 폐점한 곳 0으로 적용
for c in closed_test_store:
    y_pred[c] = 0

In [158]:
# At last the 'y_test' predicted from the trained model is adjusted with 'feb_march'.
#
# The last number 0.72 is an optimized number variable to make sure that no prediction exceeds the answer.
# This is required as penalties are given.

In [159]:
for x in range(200):
    y_pred[x] = y_pred[x] * feb_march[x] * 0.72

In [160]:

score = 13696.49609
submission['total_sales'] = y_pred

today = datetime.datetime.today().date()
os.makedirs('./submission/',exist_ok=True)
submission.to_csv(f'./submission/submission_{today}_xgb_{score}_adjust.csv',index=False)